# Prepare Data From CSV Recording

## Load File

Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.

The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.

The columns are reorded in the order of head/r_controller/l_controller/waist/r_foot/l_foot.

The new trimmed file is written to a directory (/test_data or /train_data), for further manipulation and loading into the model.


In [21]:
import pandas as pd


#Read in CSV
def GetRecording(path):
    recording_path = "../recordings/"
    file_name = input("Input Recording File Name")
    try:
        dataframe = pd.read_csv(recording_path + file_name + ".csv")
        return dataframe, file_name
    except: 
        print("Error Reading File: Check Spelling and Try Again")
        return 0
    
    
#Seperate each tracker to seperate dataframe
def GetColByName(dataframe):
    HMD = dataframe.loc[:, ["HMD0_tx", "HMD0_ty", "HMD0_tz"]]
    
    controller_1 = dataframe.loc[:, ['controller3_tx', 'controller3_ty', 'controller3_tz']]

    controller_2 = dataframe.loc[:, ['controller6_tx', 'controller6_ty', 'controller6_tz']]

    tracker_1 = dataframe.loc[:, ['generic7_tx', 'generic7_ty', 'generic7_tz']]

    tracker_2 = dataframe.loc[:, ['generic8_tx', 'generic8_ty', 'generic8_tz']]

    tracker_3 = dataframe.loc[:, ['generic9_tx', 'generic9_ty', 'generic9_tz']]

    joined = pd.concat([HMD,controller_1, controller_2, tracker_1 ,tracker_2 ,tracker_3], axis=1)
    return joined

#rename the now isolated trackers to a conventional naming scheme
def AssignTracker(dataframe):
    display(dataframe.iloc[0:1,:])
    trackerNum = int(input('Input Tracker Number'))
    for x in range(3):
        trackerStr = str(trackerNum)
        tracker = input('assign generic' + trackerStr)
        dataframe.rename(columns={'generic' + trackerStr + '_tx': tracker + '_x', 'generic' + trackerStr + '_ty': tracker + "_y", 'generic' + trackerStr + '_tz': tracker + '_z'}, inplace=True)
        trackerNum+=1
        
    controllerNum = 3
    for x in range(2):
        controllerStr = str(controllerNum)
        controller = input('assign controller' + controllerStr)
        dataframe.rename(columns={'controller' + controllerStr + '_tx': controller + '_x', 'controller' + controllerStr + '_ty': controller + "_y", 'controller' + controllerStr + '_tz': controller + '_z'}, inplace=True)
        controllerNum += 3
    dataframe.rename(columns={'HMD0_tx': 'head_x', 'HMD0_ty': 'head_y', 'HMD0_tz': 'head_z'}, inplace=True)
    return dataframe

#assign newly formatted data to test or train directory
def GetDirectory():
    choice = input("train or test data:")
    if choice == "test":
        output_path = "../test_data/"
    else:
        output_path = "../train_data/"
    return output_path

#re-order features to head / r_controller / l_controller / waist / r_foot / l_foot
def OrderFeatures(dataframe):
    head = dataframe.loc[:, ['head_x', 'head_y', 'head_z']]
    l_controller = dataframe.loc[:, ['l_controller_x', 'l_controller_y', 'l_controller_z']]
    r_controller = dataframe.loc[:, ['r_controller_x', 'r_controller_y', 'r_controller_z']]
    waist = dataframe.loc[:, ['waist_x', 'waist_y', 'waist_z']]
    r_foot = dataframe.loc[:, ['r_foot_x', 'r_foot_y', 'r_foot_z']]
    l_foot = dataframe.loc[:, ['l_foot_x', 'l_foot_y', 'l_foot_z']]
    reordered = pd.concat([head , r_controller, l_controller, waist, r_foot, l_foot], axis=1)
    return reordered

    
# write dataframe to csv
def WriteOutput(path, dataframe, filename):
    output_file = path + filename + "_trimmed.csv"
    dataframe.to_csv(output_file, index = False)
    print(file_name + " output to " + path)
    
    

# Run data trimming functions

In [29]:
recording_path = " ../recordings"
dataframe, file_name = GetRecording(recording_path)
joined = GetColByName(dataframe)
renamed = AssignTracker(joined)
path = GetDirectory()
reordered = OrderFeatures(renamed)
WriteOutput(path, reordered, file_name)


Input Recording File Name walking_12


,HMD0_tx,HMD0_ty,HMD0_tz,controller3_tx,controller3_ty,controller3_tz,controller6_tx,controller6_ty,controller6_tz,generic7_tx,generic7_ty,generic7_tz,generic8_tx,generic8_ty,generic8_tz,generic9_tx,generic9_ty,generic9_tz
0,31.318401,160.327911,63.624634,17.811829,75.635979,84.311546,36.57394,84.524063,43.69986,30.542147,103.656723,67.766724,55.069244,11.733055,57.367538,31.262779,8.882833,80.517052


Input Tracker Number 7
assign generic7 waist
assign generic8 r_foot
assign generic9 l_foot
assign controller3 l_controller
assign controller6 r_controller
train or test data: train


walking_12 output to ../train_data/


# Data Normalization

## Data Scaling

The new CSV is loaded into memory, chosen through an input prompt
The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).
These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"


#read in formatted CSV
def ReadCSV(path):
    file_name = input("Input File Name")
    file_list = []
    file_list.append(file_name)
    try:
        dataframe = pd.read_csv(path + file_name + ".csv")
        print("Dataframe created")
    except:
        print("Error Reading File")
    return dataframe, file_list

def SplitFeaturesLabels(dataframe):
    x = dataframe.iloc[:, 0:9]
    y = dataframe.iloc[:, 9:18]
    return x, y

#Load data into Numpy array
def LoadArray(x, y):
    x_array = np.array(x)
    y_array = np.array(y)
    return x_array, y_array


def NormalizeValues (x, y):
    x =  np.divide(x, 100)
    y =  np.divide(y, 100)
    return x, y

def SampleSize(x, y):
    x_samples = x[0:600,:]
    y_samples = y[0:600,:]
    return x_samples, y_samples

def RoundValues(x, y): 
    x_rounded = np.around(x, 3)
    y_rounded = np.around(y, 3)
    return x_rounded, y_rounded



    
    

    





# Create Training Data

In [2]:
train_path = "../train_data/"
#load train data from csv
train_dataframe, files = ReadCSV(train_path)

print(files)
#split features and labels into seperate dataframes
x_train_df, y_train_df = SplitFeaturesLabels(train_dataframe)

#convert features and labels to numpy array
x_train, y_train = LoadArray(x_train_df, y_train_df)

#Divide values in array by 100
x_samples, y_samples = NormalizeValues(x_train, y_train)

print(x_samples.shape, x_samples)
print(y_samples.shape, y_samples)

#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)





KeyboardInterrupt: Interrupted by user

# Scale Values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

print(x_samples.max())
print(x_samples.min())

print(y_samples.max())
print(y_samples.min())

scaler =MinMaxScaler()
print(x_samples[0:1])
scaled = scaler.fit(x_samples)
print(scaler.transform(x_samples[0:1]))
print(scaler.inverse_transform(x_samples[0:1]))


# Reshape Training Data

In [7]:

def ReshapeData(x, y):
    x_reshaped = np.expand_dims(x, axis=1)
    y_reshaped = np.expand_dims(y, axis=1)

    return x_reshaped, y_reshaped


In [68]:
x, y = ReshapeData(x_samples, y_samples)

print(x.shape, y.shape)

print(x.shape[1])

print(x.shape[2])


(600, 1, 9) (600, 1, 9)
1
9


# Create Test / Validation Data

In [32]:
#Create a single test data file

test_path = "../test_data/"


test_dataframe = ReadCSV(test_path)

#split features and labels into seperate dataframes
x_test_df, y_test_df = SplitFeaturesLabels(test_dataframe)

#convert features and labels to numpy array
x_test, y_test = LoadArray(x_test_df, y_test_df)

#Divide values in array by 100
x_test_normalized, y_test_normalized = NormalizeValues(x_test, y_test)

x_test_samples, y_test_samples = SampleSize(x_test_normalized, y_test_normalized)

x, y = ReshapeData(x_test_samples, y_test_samples)

print(x.shape, y_test.shape)

print(x_test.shape[1])

KeyboardInterrupt: Interrupted by user

# Combine Test and Train datasets

Combine all data sets in /train_data and /test_data into one, for more samples when training 

In [3]:
#import modules 

import os
import pandas as pd
import numpy as np


In [4]:

#Combine all datasets in a directory into one dataframe
def CombineDatasets(path):
    data_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            
            data_list.append(df)
         
    
    data_df = pd.concat(data_list, axis=0, ignore_index=True)
    display(data_df)

    return data_df



In [5]:

#combine data in a directory into two lists of x and y features
def DatasetsLists(path):
    x_list = []
    y_list = []
    file_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".csv"):
            file_list.append(filename)
            df = pd.read_csv(path + filename)
            x_features, y_features = SplitFeaturesLabels(df)
            x_train, y_train = LoadArray(x_features, y_features)
            x_normalized, y_normalized = NormalizeValues(x_train, y_train)
            x_reshape, y_reshape = ReshapeData(x_normalized, y_normalized)
            x_list.append(x_reshape)
            y_list.append(y_reshape)
    return x_list, y_list, file_list



In [8]:
train_path = "../train_data/"
x, y, files = DatasetsLists(train_path)


.ipynb_checkpoints
arm_raise_1_trimmed.csv
arm_raise_3_trimmed.csv
arm_raise_4_trimmed.csv
beat_saber_1_trimmed.csv
boxing_1_trimmed.csv
boxing_3_trimmed.csv
browsing_menu_1_trimmed.csv
crouching_1_trimmed.csv
crouch_walking_1_trimmed.csv
crouch_walking_3_trimmed.csv
dancing_1_trimmed.csv
dancing_2_trimmed.csv
fps_1_trimmed.csv
high_kicks_1_trimmed.csv
inspecting_object_1_trimmed.csv
interacting_1_trimmed.csv
interacting_2_trimmed.csv
jumping_1_trimmed.csv
jumping_2_trimmed.csv
jumping_3_trimmed.csv
jumping_5_trimmed.csv
jumping_to_spot_1_trimmed.csv
karate_kick_1_trimmed.csv
leg_raise_1_trimmed.csv
leg_raise_2_trimmed.csv
leg_raise_3_trimmed.csv
leg_raise_4_trimmed.csv
leg_raise_6_trimmed.csv
lunges_1_trimmed.csv
lunges_2_trimmed.csv
picking_up_1_trimmed.csv
picking_up_object_1_trimmed.csv
quick_draw_1_trimmed.csv
running_1_trimmed.csv
running_2_trimmed.csv
running_on_spot_2_trimmed.csv
running_on_spot_3_trimmed.csv
running_on_spot_4_trimmed.csv
sitting_on_floor_1_trimmed.csv
sitting_

Combine all data in the training data directory 

In [35]:
train_path = "../train_data/"

combined_train_dataframe = CombineDatasets(train_path)

x_train, y_train = SplitFeaturesLabels(combined_train_dataframe)
        
x_train_arr, y_train_arr = LoadArray(x_train, y_train)

#Divide values in array by 100
x_train_normalized, y_train_normalized = NormalizeValues(x_train_arr, y_train_arr)




x_train, y_train = ReshapeData(x_train_normalized, y_train_normalized)

print(x_train.shape, y_train.shape)


,head_x,head_y,head_z,r_controller_x,r_controller_y,r_controller_z,l_controller_x,l_controller_y,l_controller_z,waist_x,waist_y,waist_z,r_foot_x,r_foot_y,r_foot_z,l_foot_x,l_foot_y,l_foot_z,r_controller_1_z
0,-1.512766,157.191620,-9.888399,21.191143,79.166565,-11.377203,-26.088011,76.942291,-10.149074,-0.445890,96.929459,-8.480424,23.448992,9.543789,1.496696,-20.655287,9.259642,0.614011,NaN
1,-1.512766,157.191620,-9.888399,21.149824,79.149239,-11.353464,-26.017019,76.955322,-10.172288,-0.440914,96.923447,-8.475275,23.455013,9.543789,1.496696,-20.653749,9.257123,0.614011,NaN
2,-1.513591,157.191620,-9.887324,21.090712,79.145103,-11.338286,-25.940933,76.964912,-10.187013,-0.486888,96.952217,-8.502438,23.455399,9.545863,1.496696,-20.651741,9.255754,0.614011,NaN
3,-1.516653,157.191620,-9.885753,21.030048,79.139610,-11.326243,-25.873682,76.988953,-10.210484,-0.463874,96.927620,-8.485781,23.447699,9.557478,1.496696,-20.649475,9.262537,0.614011,NaN
4,-1.519660,157.191620,-9.884203,20.970465,79.133156,-11.313913,-25.790752,76.984436,-10.212699,-0.518387,96.974091,-8.520906,23.447878,9.561661,1.496696,-20.643440,9.268540,0.613830,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22103,-20.592812,153.837906,8.962499,14.495128,80.638947,24.875628,-13.531744,129.267471,5.215367,-23.028046,101.073479,15.037213,2.031792,27.983826,7.153422,-24.704552,8.931934,19.363951,NaN
22104,-20.687576,153.791916,9.007154,14.631790,80.653931,24.739309,-13.484880,128.671585,4.909608,-23.038244,101.070518,14.981284,2.113105,27.717655,6.948289,-24.700665,8.906264,19.377184,NaN
22105,-20.760902,153.795700,9.073777,14.719493,80.704208,24.436106,-13.466956,127.951714,4.596071,-23.095915,101.141968,14.837296,2.213388,27.388773,6.844291,-24.701941,8.901129,19.377184,NaN
22106,-20.845600,153.841537,9.123213,14.739350,80.788254,24.002932,-13.473389,127.110672,4.237541,-23.074080,101.136917,14.756732,2.392010,27.004562,6.799872,-24.702768,8.895117,19.377184,NaN


(22108, 1, 9) (22108, 1, 9)


Combine all data in the test data directory

In [9]:
test_path = "../test_data/"

combined_test_dataframe = CombineDatasets(test_path)

x_test, y_test = SplitFeaturesLabels(combined_test_dataframe)
        
x_test_arr, y_test_arr = LoadArray(x_test, y_test)

x_test_normalized, y_test_normalized = NormalizeValues(x_test_arr, y_test_arr)


x_test, y_test = ReshapeData(x_test_normalized, y_test_normalized)

print(x_test.shape)
print(x_test[0:1,:,:])

.ipynb_checkpoints
arm_raise_test_trimmed.csv
boxing_test_trimmed.csv
calibration_trimmed.csv
crouching_test_trimmed.csv
jumping_test_trimmed.csv
leg_raise_test_trimmed.csv
old_test_data
reverse_leg_raise_test_trimmed.csv
running_on_spot_test_trimmed.csv
sitting_standing_test_2_trimmed.csv
sitting_standing_test_trimmed.csv
walking_on_spot_test_trimmed.csv
walking_test_2_trimmed.csv
walking_test_3_trimmed.csv
walking_test_4_trimmed.csv


,head_x,head_y,head_z,r_controller_x,r_controller_y,r_controller_z,l_controller_x,l_controller_y,l_controller_z,waist_x,waist_y,waist_z,r_foot_x,r_foot_y,r_foot_z,l_foot_x,l_foot_y,l_foot_z
0,-16.908730,159.805878,18.193317,0.511658,85.282448,6.662142,-39.918804,84.929611,21.609974,-19.276501,102.381660,16.807943,12.034541,14.876402,23.484051,-37.990547,9.714794,34.886623
1,-16.929470,159.818604,18.194321,0.227490,85.347298,6.595823,-39.978241,84.929611,21.657539,-19.242987,102.358887,16.839308,12.034866,14.873164,23.484051,-37.986008,9.709342,34.886623
2,-16.935987,159.840225,18.200331,-0.072614,85.339668,6.581916,-40.010780,84.880081,21.715462,-19.173140,102.336899,16.878036,12.036016,14.864131,23.484051,-37.973251,9.698031,34.886623
3,-16.954140,159.869995,18.215652,-0.436614,85.269768,6.719634,-40.009499,84.827087,21.772154,-19.136368,102.322899,16.910797,12.037182,14.859715,23.488161,-37.965683,9.692083,34.886623
4,-16.964651,159.902878,18.213621,-0.795858,85.178627,6.884759,-39.970985,84.726952,21.813499,-19.082932,102.310738,16.940609,12.038342,14.855817,23.492746,-37.960934,9.684858,34.886623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,-43.551422,159.213654,25.716154,-41.577457,76.150658,0.517673,-44.447037,76.871948,49.416439,-40.623920,100.984367,22.470478,-55.773060,22.440622,15.880060,-20.090252,10.954879,22.744732
8396,-44.375736,159.188614,25.836290,-42.272049,75.962738,0.927628,-45.402927,77.032722,49.702805,-41.362316,100.836555,22.863989,-56.971157,22.482927,16.922010,-20.136276,10.952642,22.806454
8397,-45.221741,159.161789,25.959261,-43.040432,75.771469,1.353317,-46.321724,77.172806,49.977688,-42.111233,100.693710,23.261139,-59.270138,22.017773,20.194857,-20.182709,10.951209,22.881817
8398,-46.065994,159.139679,26.084253,-43.871861,75.583832,1.785565,-47.210728,77.307892,50.239239,-42.894726,100.543198,23.637917,-59.841599,21.764933,21.258841,-20.215130,10.951209,22.959131


(8400, 1, 9)
[[[-0.1690873   1.59805878  0.18193317  0.00511658  0.85282448
    0.06662142 -0.39918804  0.84929611  0.21609974]]]


# Model Creation and Training

In [10]:
#import required modules
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
import tensorflow as tf
from keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import SGD
from keras.layers.core import Dense, Activation, Dropout
from keras.losses import SparseCategoricalCrossentropy

tf.keras.backend.set_floatx('float64')


# Model Functions

In [11]:
#Fit model to two equally sized lists of x features and y labels 

def FitToList(x, y, model, epoch, b_size, verbose):
    for i in range(len(x)):
        print(i)
        print(x[i].shape)
        print(y[i].shape)
        model.fit(x[i], y[i], epochs=epoch,batch_size=b_size, verbose=verbose)
    return epoch, b_size
        
 # fit the model to a given set of features (x) and labels (y)    
def FitModel(x, y, model, epoch, b_size):
    model.fit(x, y, validation_split=0.33, epochs=epoch,batch_size = b_size)
    return epoch, b_size

    
def EvaluateModel(x, y, model):
    metrics = model.evaluate(x, y, batch_size=256)
    print("test loss, test acc:", metrics)
    return metrics
    
    
def PredictModel(x, y, model):
    predictions = model.predict(x)
    y_reshaped = y.reshape(-1,9)
    x_reshaped = x.reshape(-1,9)
    return predictions, y_reshaped, x_reshaped


def DisplayPredictions(prediction, actual, predicted_on, range_1, range_2):
    if range_2 <= 0:
        r_1 = 0
        r_2 = len(prediction)
    else:
        r_1 = range_1
        r_2 = range_2
    print("predictions shape:", prediction.shape)
    predicted_on_df = pd.DataFrame(predicted_on, columns=["Head_X", "Head_Y", "Head_Z", "R_Controller_X", "R_Controller_Y", "R_Controller_Z", "L_Controller_X", "L_Controller_Y", "L_Controller_Z"])
    prediction_df = pd.DataFrame(prediction, columns=["Waist_X", "Waist_Y", "Waist_Z", "R_Foot_X", "R_Foot_Y", "R_Foot_Z", "L_Foot_X", "L_Foot_Y", "L_Foot_Z"])
    actual_df = pd.DataFrame(actual, columns=["Waist_X", "Waist_Y", "Waist_Z", "R_Foot_X", "R_Foot_Y", "R_Foot_Z", "L_Foot_X", "L_Foot_Y", "L_Foot_Z"])
    print("Actual Values")
    display(actual_df[r_1:r_2])
    print("Predicited Values")
    display(prediction_df[r_1:r_2])
    return actual_df, prediction_df, predicted_on_df
    

        

# Model 1

In [8]:
sgd = SGD(learning_rate=0.001, momentum=0.8, decay=0.999, nesterov=False)

tf.keras.backend.set_floatx('float64')

model = Sequential()
model.add(GRU(16, return_sequences=True, input_shape=(5, 9)))
model.add(Dropout(0.2))
model.add(GRU(16, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(9, activation = "linear"))

model.compile(loss='mse', optimizer="adam")

print ('model compiled')

print (model.summary())

model compiled
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 16)             1296      
                                                                 
 dropout (Dropout)           (None, 1, 16)             0         
                                                                 
 gru_1 (GRU)                 (None, 16)                1632      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 9)                 153       
                                                                 
Total params: 3,081
Trainable params: 3,081
Non-trainable params: 0
_________________________________________________________________
None


# Model 2 

In [38]:
model_2 = Sequential()

opt = SGD(learning_rate = 0.01)

compile_params = ["mse", "sgd", "accuracy"]
#add one GRU layer of 64 cellss with input shape 1,9
model_2.add(GRU(64,return_sequences=True, input_shape=(1, 9)))

#add dropout layer
model_2.add(Dropout(0.2))

#add second gru layer 
model_2.add(GRU(units=64))


#Add Dense layer with 9 outputs
model_2.add(Dense(9))

print(model_2.summary())


#Compile model
model_2.compile(
    loss = compile_params[0],
    optimizer = opt,
    metrics = [compile_params[2]],
)


print(model_2.compiled_loss._losses)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_6 (GRU)                 (None, 1, 64)             14400     
                                                                 
 dropout_3 (Dropout)         (None, 1, 64)             0         
                                                                 
 gru_7 (GRU)                 (None, 64)                24960     
                                                                 
 dense_3 (Dense)             (None, 9)                 585       
                                                                 
Total params: 39,945
Trainable params: 39,945
Non-trainable params: 0
_________________________________________________________________
None
mse


In [13]:
epochs, batch_size = FitModel(x, y, model_2, 1, 256)

ValueError: Data cardinality is ambiguous:
  x sizes: 606, 600, 600, 600, 600, 600, 600, 600, 606, 600, 600, 600, 606, 600, 600, 600, 600, 600, 600, 600, 606, 600, 600, 600, 600, 600, 606, 606, 600, 600, 600, 600, 600
  y sizes: 606, 600, 600, 600, 600, 600, 600, 600, 606, 600, 600, 600, 606, 600, 600, 600, 600, 600, 600, 600, 606, 600, 600, 600, 600, 600, 606, 606, 600, 600, 600, 600, 600
Make sure all arrays contain the same number of samples.

In [39]:
#Fit list of datasets to model

#training_x, training_y, model to train on, epoch count, batch size, ?verbose
epochs, batch_size = FitToList(x, y, model_2, 20, 1, 1)

0
(905, 1, 9)
(905, 1, 9)
Epoch 1/20
905/905 [==============================] - 4s 2ms/step - loss: 0.0138 - accuracy: 0.9867
Epoch 2/20
905/905 [==============================] - 2s 2ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 3/20
905/905 [==============================] - 2s 2ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 4/20
905/905 [==============================] - 2s 2ms/step - loss: 8.7689e-04 - accuracy: 1.0000
Epoch 5/20
905/905 [==============================] - 2s 2ms/step - loss: 8.2814e-04 - accuracy: 1.0000
Epoch 6/20
905/905 [==============================] - 2s 2ms/step - loss: 7.5409e-04 - accuracy: 1.0000
Epoch 7/20
905/905 [==============================] - 2s 2ms/step - loss: 7.3486e-04 - accuracy: 1.0000
Epoch 8/20
905/905 [==============================] - 2s 2ms/step - loss: 6.8019e-04 - accuracy: 1.0000
Epoch 9/20
905/905 [==============================] - 2s 2ms/step - loss: 6.1335e-04 - accuracy: 1.0000
Epoch 10/20
905/905 [=============================

In [40]:

results = EvaluateModel(x_test, y_test, model_2)

33/33 [==============================] - 1s 5ms/step - loss: 0.0431 - accuracy: 0.9232
test loss, test acc: [0.04307476711415109, 0.9232149264442052]


In [41]:
predicted_values, actual_values, predicted_on_values = PredictModel(x_test, y_test, model_2)

toWriteActual, toWritePred, toWritePredictedOn = DisplayPredictions(predicted_values, actual_values, predicted_on_values, 0, 0)

display(toWritePredictedOn)


predictions shape: (8400, 9)
Actual Values


,Waist_X,Waist_Y,Waist_Z,R_Foot_X,R_Foot_Y,R_Foot_Z,L_Foot_X,L_Foot_Y,L_Foot_Z
0,-0.192765,1.023817,0.168079,0.120345,0.148764,0.234841,-0.379905,0.097148,0.348866
1,-0.192430,1.023589,0.168393,0.120349,0.148732,0.234841,-0.379860,0.097093,0.348866
2,-0.191731,1.023369,0.168780,0.120360,0.148641,0.234841,-0.379733,0.096980,0.348866
3,-0.191364,1.023229,0.169108,0.120372,0.148597,0.234882,-0.379657,0.096921,0.348866
4,-0.190829,1.023107,0.169406,0.120383,0.148558,0.234927,-0.379609,0.096849,0.348866
...,...,...,...,...,...,...,...,...,...
8395,-0.406239,1.009844,0.224705,-0.557731,0.224406,0.158801,-0.200903,0.109549,0.227447
8396,-0.413623,1.008366,0.228640,-0.569712,0.224829,0.169220,-0.201363,0.109526,0.228065
8397,-0.421112,1.006937,0.232611,-0.592701,0.220178,0.201949,-0.201827,0.109512,0.228818
8398,-0.428947,1.005432,0.236379,-0.598416,0.217649,0.212588,-0.202151,0.109512,0.229591


Predicited Values


,Waist_X,Waist_Y,Waist_Z,R_Foot_X,R_Foot_Y,R_Foot_Z,L_Foot_X,L_Foot_Y,L_Foot_Z
0,-0.172490,1.010595,0.196057,0.019770,0.152579,0.218240,-0.296400,0.141983,0.250397
1,-0.173291,1.010664,0.195882,0.018948,0.152570,0.217969,-0.297235,0.142034,0.250008
2,-0.174022,1.010743,0.195816,0.018127,0.152568,0.217827,-0.297981,0.142064,0.249664
3,-0.175097,1.010861,0.196013,0.016863,0.152672,0.217914,-0.299022,0.142006,0.249459
4,-0.176068,1.010991,0.196125,0.015623,0.152769,0.217961,-0.299985,0.141928,0.249144
...,...,...,...,...,...,...,...,...,...
8395,-0.374169,1.011751,0.247610,-0.143570,0.167753,0.230534,-0.442567,0.122830,0.283073
8396,-0.382584,1.011790,0.249641,-0.150210,0.168917,0.231495,-0.449091,0.121628,0.286258
8397,-0.391211,1.011818,0.251623,-0.157134,0.170108,0.232371,-0.455816,0.120389,0.289276
8398,-0.399919,1.011853,0.253532,-0.164219,0.171304,0.233159,-0.462649,0.119149,0.292108


,Head_X,Head_Y,Head_Z,R_Controller_X,R_Controller_Y,R_Controller_Z,L_Controller_X,L_Controller_Y,L_Controller_Z
0,-0.169087,1.598059,0.181933,0.005117,0.852824,0.066621,-0.399188,0.849296,0.216100
1,-0.169295,1.598186,0.181943,0.002275,0.853473,0.065958,-0.399782,0.849296,0.216575
2,-0.169360,1.598402,0.182003,-0.000726,0.853397,0.065819,-0.400108,0.848801,0.217155
3,-0.169541,1.598700,0.182157,-0.004366,0.852698,0.067196,-0.400095,0.848271,0.217722
4,-0.169647,1.599029,0.182136,-0.007959,0.851786,0.068848,-0.399710,0.847270,0.218135
...,...,...,...,...,...,...,...,...,...
8395,-0.435514,1.592137,0.257162,-0.415775,0.761507,0.005177,-0.444470,0.768719,0.494164
8396,-0.443757,1.591886,0.258363,-0.422720,0.759627,0.009276,-0.454029,0.770327,0.497028
8397,-0.452217,1.591618,0.259593,-0.430404,0.757715,0.013533,-0.463217,0.771728,0.499777
8398,-0.460660,1.591397,0.260843,-0.438719,0.755838,0.017856,-0.472107,0.773079,0.502392


# Write Results to File 

In [43]:
from datetime import datetime

def WriteResultToFile(path, pred, actual, model, metrics, file_list, epochs, b_size, predicted_on):
    try:
        #get current date for folder naming
        now = datetime.now()
        dt_string = now.strftime("%d_%m_%Y_%H_%M")
    
        #create new folder
        new_folder = "run_" + dt_string
        os.makedirs(path + new_folder)

        #loop over columns to split between actual and predicted
        for column in actual.columns:
            new_column = "A_" + column
            actual = actual.rename(columns={column : new_column})
        for column in pred.columns:
            new_column = "P_" + column
            pred = pred.rename(columns={column : new_column})

        #combine both actual and predicted dataframes
        results = pd.concat([actual, pred], axis=1)
    
        #create output path for csv write
        output_folder = results_path + new_folder
    
        predicted_on.to_csv(output_folder + "/test_values.csv", index = False, float_format = '%.6f')
        results.to_csv(output_folder + "/prediction.csv", index = False, float_format='%.6f')
        metric_labels = ["Loss: ", "Accuracy: "]    
        compile_params_list = ["Loss Function: ", "Optimizer: ", "Metrics: "]
        #create new txt file to output model and training data summary
        with open(output_folder + '/summary.txt','w') as fh:
            model.summary(print_fn=lambda x: fh.write(x + '\n'))
            fh.write("Epochs: ")
            fh.write(str(epochs) + "\n")
            fh.write("Batch Size: ")
            fh.write(str(b_size) + "\n")
            fh.write("Metrics:\n")
            for i in range (len(metrics)):
                fh.write(metric_labels[i])
                fh.write(str(metrics[i]) + "\n")
            for i in range (len(compile_params)):
                fh.write(compile_params_list[i])
                fh.write(compile_params[i] + "\n")
            fh.write("Trained On:\n")
            for file in file_list:
                fh.write(file + "\n")
           
        print("File Output to " + new_folder)
    except Exception as e:
        
        print(e)
        print("Error Writing Results To File!")
        
        
        
    
    


In [44]:
results_path = "../results/"

WriteResultToFile(results_path, toWritePred, toWriteActual, model_2, results, files, epochs, batch_size, toWritePredictedOn) 



File Output to run_15_08_2022_02_28
